# LocEmb: Location Embedding

만든 이: 김선동 (http://seondong.github.io)

### 프로젝트 목표
* 전국 지역 단위별 embedding을 제공하고자 함
* 도로명, 상권, 사업자 등으로 확장 가능

### 사용 가능한 프로젝트
* 장소 추천/예측 모델에 pretrained된 embedding값을 활용
* COVID-19 확산/예측 SIR-based 모델 개발시 pretrained된 지역별 embedding값을 covariate으로 활용

### 사용한 알고리즘
* [NeurIPS2017](https://papers.nips.cc/paper/7213-poincare-embeddings-for-learning-hierarchical-representations)을 기반으로 다양한 feature를 추가하였음

### 제공 임베딩
* 시군구2vec: (251개 시군구 Cover, 100 dimensions): embedding+시군구명.csv
* 법정동2vec: (5005개 법정동 Cover, 100 dimensions): embedding+법정동명.csv
* 행정동2vec: (3582개 행정동 Cover, 100 dimensions): embedding+행정동명.csv
* 도로명2vec: (110722개 도로명 Cover, 100 dimensions): embedding+도로명.csv

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.width', 1000)
pd.options.display.max_colwidth = 100

## 1. Coarse-grained Embedding: 시군구2vec

In [2]:
emb_rslt = pd.read_csv("embedding+시군구명.csv",index_col=0)

In [3]:
emb_rslt.shape

(251, 100)

In [4]:
emb_rslt.index

Index(['서울특별시 강남구', '서울특별시 금천구', '부산광역시 사상구', '서울특별시 송파구', '서울특별시 광진구', '서울특별시 강서구', '서울특별시 중구', '서울특별시 서초구', '서울특별시 성북구', '서울특별시 구로구',
       ...
       '경상북도 고령군', '전라남도 장흥군', '전라남도 영암군', '전라남도 진도군', '전라북도 부안군', '충청남도 예산군', '경상북도 영양군', '전라북도 완주군', '전라북도 무주군', '충청남도 서천군'], dtype='object', length=251)

In [5]:
emb_rslt.loc['대전광역시 유성구']

0    -0.101524
1    -0.035251
2     0.008657
3    -0.016047
4    -0.126945
        ...   
95   -0.054820
96   -0.106948
97   -0.073350
98    0.022328
99   -0.020869
Name: 대전광역시 유성구, Length: 100, dtype: float64

### 시군구 임베딩간 유사도 체크

In [6]:
from scipy.spatial.distance import cosine
import operator

def print_top_n_similar(query, topn=10, emb_rslt=emb_rslt):
    dembs = {}
    for i in emb_rslt.index:
        dembs[i] = np.array(emb_rslt.loc[i])
    
    distance = {}
    for i in dembs:
        distance[i] = cosine(dembs[query], dembs[i])
    return sorted(distance.items(), key=operator.itemgetter(1))[:topn]

In [7]:
print_top_n_similar('대전광역시 유성구', topn=10, emb_rslt = emb_rslt)

[('대전광역시 유성구', 0.0),
 ('충청남도 계룡시', 0.44828839508161566),
 ('대전광역시 대덕구', 0.4661577359698795),
 ('대전광역시 서구', 0.5941670326674005),
 ('대구광역시 달서구', 0.7034313245577278),
 ('경상북도 영덕군', 0.74106036565266),
 ('충청북도 청주시 서원구', 0.794354636178245),
 ('경상남도 사천시', 0.7979638038050675),
 ('충청남도 공주시', 0.8208037072234458),
 ('부산광역시 기장군', 0.822934982494978)]

In [8]:
print_top_n_similar('서울특별시 강남구', topn=10, emb_rslt = emb_rslt)

[('서울특별시 강남구', 0.0),
 ('서울특별시 송파구', 0.3472763771014885),
 ('서울특별시 서초구', 0.39687958947028845),
 ('경기도 성남시 수정구', 0.727289145321818),
 ('서울특별시 용산구', 0.741566144526395),
 ('경기도 연천군', 0.7456407484330934),
 ('울산광역시 남구', 0.7550280661092346),
 ('서울특별시 광진구', 0.7626552964322859),
 ('서울특별시 강동구', 0.7681690871282211),
 ('경기도 안성시', 0.7939815780576082)]

In [9]:
print_top_n_similar('경기도 성남시 분당구', topn=10, emb_rslt = emb_rslt)

[('경기도 성남시 분당구', 0.0),
 ('경기도 성남시 중원구', 0.3115725503395338),
 ('경기도 성남시 수정구', 0.32152080841087494),
 ('경기도 용인시 수지구', 0.3559837402745458),
 ('경기도 의왕시', 0.6206607139525472),
 ('경기도 용인시 기흥구', 0.6246114392635784),
 ('경기도 과천시', 0.627098690615472),
 ('경기도 안양시 동안구', 0.7132537276961229),
 ('경기도 광주시', 0.7581819918650335),
 ('서울특별시 도봉구', 0.7634899746657331)]

## 2. Fine-grained Embedding: 법정동2vec

### Train된 vector 로드


In [10]:
emb_rslt = pd.read_csv("embedding+법정동명.csv",index_col=0)

In [11]:
emb_rslt.shape

(5005, 100)

In [12]:
emb_rslt.index

Index(['서울특별시 강남구 대치동', '서울특별시 강남구 역삼동', '서울특별시 금천구 독산동', '부산광역시 사상구 괘법동', '서울특별시 송파구 석촌동', '서울특별시 송파구 신천동', '서울특별시 광진구 구의동', '서울특별시 강서구 공항동', '서울특별시 중구 신당동', '서울특별시 강서구 화곡동',
       ...
       '전라북도 군산시 송창동', '전라남도 나주시 진포동', '경상북도 영천시 범어동', '경상남도 창원시 진해구 현동', '전라북도 익산시 은기동', '경상남도 창원시 진해구 대영동', '충청남도 홍성군 홍북읍', '전라북도 전주시 덕진구 중동', '경상남도 창원시 성산구 완암동', '경상북도 청송군 주왕산면'], dtype='object', length=5005)

In [13]:
emb_rslt.loc['대전광역시 유성구 도룡동']

0    -0.096376
1    -0.044024
2     0.028136
3    -0.030516
4    -0.135996
        ...   
95   -0.046185
96   -0.120554
97   -0.104367
98    0.037933
99   -0.015635
Name: 대전광역시 유성구 도룡동, Length: 100, dtype: float64

### 법정동 임베딩간 유사도 체크

In [14]:
def print_top_n_similar(query, topn=10, emb_rslt=emb_rslt):
    dembs = {}
    for i in emb_rslt.index:
        dembs[i] = np.array(emb_rslt.loc[i])
    
    distance = {}
    for i in dembs:
        distance[i] = cosine(dembs[query], dembs[i])
    return sorted(distance.items(), key=operator.itemgetter(1))[:topn]

In [15]:
print_top_n_similar('대전광역시 유성구 도룡동', topn=10, emb_rslt=emb_rslt)

[('대전광역시 유성구 도룡동', 0.0),
 ('대전광역시 유성구 덕진동', 0.0019398630727633925),
 ('대전광역시 유성구 가정동', 0.0022349195564489444),
 ('대전광역시 유성구 화암동', 0.0029246081381592015),
 ('대전광역시 유성구 장동', 0.0038995779631886274),
 ('대전광역시 유성구 자운동', 0.0062843491752097025),
 ('대전광역시 유성구 추목동', 0.00631647201647767),
 ('대전광역시 유성구 신성동', 0.008692643581947657),
 ('대전광역시 유성구 신봉동', 0.009264830836538396),
 ('대전광역시 유성구 관평동', 0.010451120678122372)]

In [16]:
print_top_n_similar('서울특별시 강남구 대치동', topn=10)

[('서울특별시 강남구 대치동', 0.0),
 ('서울특별시 강남구 도곡동', 0.3394210949909462),
 ('서울특별시 강남구 일원동', 0.36272846484343435),
 ('서울특별시 강남구 개포동', 0.3776008498186064),
 ('서울특별시 강남구 수서동', 0.38990768873160786),
 ('서울특별시 강남구 삼성동', 0.43657639615304866),
 ('서울특별시 강남구 자곡동', 0.43867943439845447),
 ('서울특별시 강남구 율현동', 0.45251054448294314),
 ('서울특별시 강남구 세곡동', 0.47782944433330543),
 ('서울특별시 강남구 압구정동', 0.5725527912534945)]

In [17]:
print_top_n_similar('경기도 성남시 분당구 정자동')

[('경기도 성남시 분당구 정자동', 0.0),
 ('경기도 성남시 분당구 수내동', 0.02348145435873339),
 ('경기도 성남시 분당구 궁내동', 0.05003337956643705),
 ('경기도 성남시 분당구 금곡동', 0.0529731205965005),
 ('경기도 성남시 분당구 백현동', 0.06589377455455714),
 ('경기도 성남시 분당구 삼평동', 0.08604241089820386),
 ('경기도 성남시 분당구 율동', 0.08927583136346529),
 ('경기도 성남시 분당구 판교동', 0.08967985623005181),
 ('경기도 성남시 분당구 이매동', 0.09209539680770984),
 ('경기도 성남시 분당구 하산운동', 0.09996121746790643)]

## 3. Fine-grained Embedding: 행정동2vec

### Train된 vector 로드


In [18]:
emb_rslt = pd.read_csv("embedding+행정동명.csv",index_col=0)

In [19]:
emb_rslt.shape

(3582, 100)

In [20]:
emb_rslt.index

Index(['서울특별시 강남구 대치1동 (행정동)', '서울특별시 강남구 역삼1동 (행정동)', '서울특별시 금천구 독산3동 (행정동)', '부산광역시 사상구 괘법동 (행정동)', '서울특별시 송파구 석촌동 (행정동)', '서울특별시 송파구 잠실6동 (행정동)', '서울특별시 광진구 구의2동 (행정동)', '서울특별시 강서구 공항동 (행정동)', '서울특별시 중구 약수동 (행정동)', '서울특별시 강서구 화곡1동 (행정동)',
       ...
       '충청남도 서천군 시초면 (행정동)', '경상남도 진주시 하대동 (행정동)', '경상남도 창원시 마산회원구 석전동 (행정동)', '경상남도 진주시 상대동 (행정동)', '경상남도 창원시 마산합포구 반월중앙동 (행정동)', '충청남도 홍성군 홍북읍 (행정동)', '전라북도 전주시 완산구 효자5동 (행정동)', '전라북도 전주시 덕진구 혁신동 (행정동)', '전라남도 나주시 송월동 (행정동)', '경상북도 청송군 주왕산면 (행정동)'], dtype='object', length=3582)

In [21]:
emb_rslt.loc['대전광역시 유성구 신성동 (행정동)']

0    -0.096121
1    -0.033550
2     0.030865
3    -0.036145
4    -0.123113
        ...   
95   -0.054902
96   -0.133306
97   -0.092315
98    0.036808
99   -0.016143
Name: 대전광역시 유성구 신성동 (행정동), Length: 100, dtype: float64

### 행정동 임베딩간 유사도 체크

In [22]:
def print_top_n_similar(query, topn=10, emb_rslt=emb_rslt):
    dembs = {}
    for i in emb_rslt.index:
        dembs[i] = np.array(emb_rslt.loc[i])
    
    distance = {}
    for i in dembs:
        distance[i] = cosine(dembs[query], dembs[i])
    return sorted(distance.items(), key=operator.itemgetter(1))[:topn]

In [23]:
print_top_n_similar('대전광역시 유성구 신성동 (행정동)', topn=10, emb_rslt=emb_rslt)

[('대전광역시 유성구 신성동 (행정동)', 0.0),
 ('대전광역시 유성구 관평동 (행정동)', 0.021876420101391902),
 ('대전광역시 유성구 전민동 (행정동)', 0.026652211291311123),
 ('대전광역시 유성구 구즉동 (행정동)', 0.039132064550121526),
 ('대전광역시 유성구 온천2동 (행정동)', 0.05054110833818537),
 ('대전광역시 서구 만년동 (행정동)', 0.06536609827122164),
 ('대전광역시 유성구 노은2동 (행정동)', 0.09147975593910274),
 ('대전광역시 유성구 노은1동 (행정동)', 0.09243195599407739),
 ('대전광역시 유성구 노은3동 (행정동)', 0.11486752451741611),
 ('대전광역시 유성구 온천1동 (행정동)', 0.18604340540005115)]

In [24]:
print_top_n_similar('경기도 성남시 수정구 신흥1동 (행정동)', topn=10)

[('경기도 성남시 수정구 신흥1동 (행정동)', 0.0),
 ('경기도 성남시 수정구 신흥3동 (행정동)', 0.005042177055988084),
 ('경기도 성남시 수정구 신흥2동 (행정동)', 0.01880039582660631),
 ('경기도 성남시 수정구 산성동 (행정동)', 0.21619363328121055),
 ('경기도 성남시 수정구 단대동 (행정동)', 0.3215224639761911),
 ('경기도 성남시 수정구 양지동 (행정동)', 0.34316624079800806),
 ('경기도 성남시 중원구 중앙동 (행정동)', 0.3808286109575636),
 ('경기도 성남시 수정구 복정동 (행정동)', 0.4310022250990764),
 ('경기도 성남시 수정구 위례동 (행정동)', 0.43975653964561656),
 ('경기도 성남시 수정구 태평4동 (행정동)', 0.439891314399833)]

In [25]:
print_top_n_similar('광주광역시 서구 상무1동 (행정동)', topn=10)

[('광주광역시 서구 상무1동 (행정동)', 0.0),
 ('광주광역시 서구 상무2동 (행정동)', 0.017728288090380406),
 ('광주광역시 서구 치평동 (행정동)', 0.07321159011658962),
 ('광주광역시 서구 유덕동 (행정동)', 0.08256232448529333),
 ('광주광역시 서구 화정1동 (행정동)', 0.188840218960174),
 ('광주광역시 서구 동천동 (행정동)', 0.21308216177710748),
 ('광주광역시 서구 서창동 (행정동)', 0.23751348834131336),
 ('광주광역시 서구 금호1동 (행정동)', 0.28738025535827094),
 ('광주광역시 서구 금호2동 (행정동)', 0.3197941227357268),
 ('광주광역시 서구 광천동 (행정동)', 0.3271487928529496)]

## 4. Fine-grained Embedding: 도로명2vec

### Train된 vector 로드


In [26]:
emb_rslt = pd.read_csv("embedding+도로명.csv",index_col=0)

In [27]:
emb_rslt.shape

(110722, 100)

In [28]:
emb_rslt.index

Index(['서울특별시 강남구 남부순환로', '서울특별시 강남구 선릉로93길', '서울특별시 금천구 가산로', '부산광역시 사상구 괘감로', '서울특별시 송파구 가락로', '서울특별시 송파구 올림픽로', '서울특별시 광진구 자양로39길', '서울특별시 강서구 공항대로3길', '서울특별시 중구 동호로7길', '서울특별시 강서구 화곡로18길',
       ...
       '충청남도 서산시 학동10로', '전라북도 전주시 덕진구 가인로', '경상북도 포항시 북구 흥해읍 성곡서로18번길', '경상남도 합천군 율곡면 노양길', '경상남도 사천시 곤양면 고동포로', '전라남도 여수시 소라면 대곡해안길', '전라남도 여수시 문수북3길', '충청남도 당진시 송산면 서정6길', '충청남도 논산시 연산면 선비로646번길', '경상북도 예천군 호명면 새움1로'], dtype='object', length=110722)

In [29]:
emb_rslt.loc['대전광역시 유성구 엑스포로']

0    -0.098625
1    -0.054827
2     0.017174
3    -0.027535
4    -0.140265
        ...   
95   -0.034581
96   -0.096495
97   -0.116205
98    0.030405
99   -0.018321
Name: 대전광역시 유성구 엑스포로, Length: 100, dtype: float64

### 도로명 임베딩간 유사도 체크

In [30]:
def print_top_n_similar(query, topn=10, emb_rslt=emb_rslt):
    dembs = {}
    for i in emb_rslt.index:
        dembs[i] = np.array(emb_rslt.loc[i])
    
    distance = {}
    for i in dembs:
        distance[i] = cosine(dembs[query], dembs[i])
    return sorted(distance.items(), key=operator.itemgetter(1))[:topn]

In [31]:
print_top_n_similar('서울특별시 강남구 테헤란로', topn=10, emb_rslt=emb_rslt)

[('서울특별시 강남구 테헤란로', 0.0),
 ('서울특별시 강남구 테헤란로47길', 0.0275179394559677),
 ('서울특별시 강남구 선릉로91길', 0.0323453812932748),
 ('서울특별시 강남구 역삼로', 0.03452905695115527),
 ('서울특별시 강남구 선릉로99길', 0.04540327110712772),
 ('서울특별시 강남구 테헤란로55길', 0.05343564393502509),
 ('서울특별시 강남구 테헤란로51길', 0.06225098309234056),
 ('서울특별시 강남구 테헤란로57길', 0.06635382499816112),
 ('서울특별시 강남구 선릉로90길', 0.06962670048208297),
 ('서울특별시 강남구 봉은사로', 0.07034061660169955)]

In [32]:
print_top_n_similar('부산광역시 사하구 낙동대로', topn=10, emb_rslt=emb_rslt)

[('부산광역시 사하구 낙동대로', 0.0),
 ('부산광역시 사하구 승학로', 0.015802638494118337),
 ('부산광역시 사하구 제석로18번길', 0.01655808705211581),
 ('부산광역시 사하구 동매로', 0.018345108305352964),
 ('부산광역시 사하구 괴정로', 0.02117502532938831),
 ('부산광역시 사하구 제석로', 0.024434767585734818),
 ('부산광역시 사하구 낙동대로436번길', 0.028940565138159124),
 ('부산광역시 사하구 낙동대로407번길', 0.03214763564723899),
 ('부산광역시 사하구 낙동대로410번길', 0.034144826602908185),
 ('부산광역시 사하구 승학로17번길', 0.038548895601949296)]

In [33]:
print_top_n_similar('대전광역시 유성구 엑스포로', topn=10)

[('대전광역시 유성구 엑스포로', 0.0),
 ('대전광역시 유성구 문지로', 0.0002718513287518487),
 ('대전광역시 유성구 전민로', 0.00038442923126769113),
 ('대전광역시 유성구 엑스포로540번길', 0.0007245345632788114),
 ('대전광역시 유성구 엑스포로466번길', 0.0008326375000698993),
 ('대전광역시 유성구 엑스포로538번길', 0.0008356208308727409),
 ('대전광역시 유성구 유성대로1689번길', 0.0008854344348454557),
 ('대전광역시 유성구 엑스포로446번길', 0.0008893499329751808),
 ('대전광역시 유성구 엑스포로295번길', 0.0009518559055993192),
 ('대전광역시 유성구 유성대로1628번길', 0.001476171812361371)]